In [60]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bertopic import BERTopic

In [61]:
# import the preprocess doc

df = pd.read_csv('./Data_csv/data_preprocessed.csv')

In [62]:
df.columns

Index(['doc_id', 'Name of the document', 'Institution', 'URL', 'Authors',
       'Affiliates', 'Sector', 'Country', 'Date', 'Keywords',
       'Exclusion criteria', 'Status', 'Label', 'MapAIE (ours)', 'Jobin',
       'Fjeld', 'Tidjon', 'Hagendorff', 'Floridi', 'Zeng (LAIP)',
       'Attard-Frost', 'EP', 'Algorithm watch', 'CE', 'Winfield',
       'EthicalML GitHub', 'all sources', 'Checked by', 'Unnamed: 27',
       'Unnamed: 28', 'Unnamed: 29', 'text', 'langue', 'text_processed',
       'tfidf', 'categorie Institution', 'theme'],
      dtype='object')

In [68]:
df.count()

doc_id                   47
Name of the document     47
Institution              47
URL                      47
Authors                   6
Affiliates                5
Sector                    8
Country                  45
Date                      8
Keywords                  0
Exclusion criteria        0
Status                   47
Label                    39
MapAIE (ours)            47
Jobin                     6
Fjeld                     4
Tidjon                    3
Hagendorff                1
Floridi                   1
Zeng (LAIP)               7
Attard-Frost              5
EP                        2
Algorithm watch          13
CE                       39
Winfield                  1
EthicalML GitHub          1
all sources              47
Checked by               47
Unnamed: 27               0
Unnamed: 28               0
Unnamed: 29               2
text                     47
langue                   47
text_processed           47
tfidf                    47
categorie Institutio

In [63]:
df['text_processed'][0]

"ethics canvas brainstorm group ethical implications project represent canvas analyse ethical concerns project find suitable solutions join community learn similar use cases share experiences try canvas online start using online ethics canvas without registration export work pdf json format try online sign save canvas create account save work dashboard sign log new password name email password repeat password sign email password forgot password log email reset password download pdf version work offline download printable version ethics canvas use workshops need help download handbook details examples download pdf works ethics canvas helps structure ideas ethical implications projects working visualise resolve add cards ideas fields canvas drag drop cards different fields link cards instance specific stakeholders specific ethical concerns canvas dynamic adapt workflow ’ get overview prominent ethical concerns project glancing canvas complicated setup configuration required plug play eth

In [64]:
topic_model = BERTopic(language="english", calculate_probabilities=True, min_topic_size=8, nr_topics=10)

In [65]:
documents = df['text_processed'].to_list()

In [66]:
topics, probs = topic_model.fit_transform(documents)

In [67]:
# Afficher les topics les plus fréquents
print(topic_model.get_topic_info())



   Topic  Count                                Name  \
0     -1      4  -1_important_width_awards_fontsize   
1      0      9                0_must_ai_call_human   
2      1     34   1_ai_intelligence_artificial_data   

                                      Representation  \
0  [important, width, awards, fontsize, mainsecon...   
1  [must, ai, call, human, rome, ethic, cm, coe, ...   
2  [ai, intelligence, artificial, data, systems, ...   

                                 Representative_Docs  
0  [html height:100 body margin:0 height:100, pdf...  
1  [coe search cm, rome call ai ethic rome call a...  
2  [ai principles telefonica approved executive c...  
